In [ ]:
abspath = '/Users/paolamartire/shocks'
import sys
sys.path.append(abspath)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import healpy as hp
from Utilities.basic_units import radians
from src import orbits as orb
from Utilities import sections as sec
from Utilities.sections import make_slices
import Utilities.prelude as prel
from sklearn.neighbors import KDTree
from sympy.vector import CoordSys3D, curl
from Utilities.selectors_for_snap import select_prefix
from Utilities.operators import make_tree, to_spherical_components, choose_sections, to_spherical_coordinate, draw_line

In [ ]:
m = 4
Mbh = 10**m
beta = 1
mstar = .5
Rstar = .47
n = 1.5 # 'n1.5'
gamma = 5/3
params = [Mbh, Rstar, mstar, beta]
check = 'HiResNewAMR' # '' or 'HiRes' or 'LowRes'
snap = '76'
compton = 'Compton'

folder = f'R{Rstar}M{mstar}BH{Mbh}beta{beta}S60n{n}{compton}{check}' 
path = select_prefix(m, check, mstar, Rstar, beta, n, compton)
path = f'{path}/{snap}'

params = [Mbh, Rstar, mstar, beta]
things = orb.get_things_about(params)
Rs = things['Rs']
Rt = things['Rt']
Rp = things['Rp']
R0 = things['R0']
apo = things['apo']
a_mb = things['a_mb']
E_mb = things['E_mb']
t_fb_days_cgs = things['t_fb_days'] * 24 * 3600 # in seconds
v_esc = np.sqrt(2*prel.G*Mbh/Rp)

# cfr tidal disruption and at smoothing lenght
xcfr, ycfr, cfr = orb.make_cfr(Rt)
xcfr0, ycfr0, cfr0 = orb.make_cfr(R0)
xcfr05mb, ycfr05mb, cfr05mb = orb.make_cfr(0.5*a_mb)

# HEALPIX
observers_xyz = hp.pix2vec(prel.NSIDE, range(prel.NPIX))
observers_xyz = np.array(observers_xyz).T
x, y, z = observers_xyz[:, 0], observers_xyz[:, 1], observers_xyz[:, 2]
r = np.sqrt(x**2 + y**2 + z**2)   # Radius (should be 1 for unit vectors)
theta = np.arctan2(y, x)          # Azimuthal angle in radians
phi = np.arccos(z / r)            # Elevation angle in radians
longitude_moll = theta              
latitude_moll = np.pi / 2 - phi 
indecesorbital = np.concatenate(np.where(latitude_moll==0))
first_idx, last_idx = np.min(indecesorbital), np.max(indecesorbital)

Load data

In [ ]:
tfb = np.loadtxt(f'{path}/tfb_{snap}.txt')
# DivV = np.load(f'{path}/DivV_{snap}.npy')
data = make_tree(path, snap, energy = True)
cut = data.Den > 1e-19
X, Y, Z, mass, den, Vol, Mass, Press, Temp, vx, vy, vz, IE_den, Diss_den, Rad_den = \
    make_slices([data.X, data.Y, data.Z, data.Mass, data.Den, data.Vol, data.Mass, data.Press, data.Temp, data.VX, data.VY, data.VZ, data.IE, data.Diss, data.Rad], cut)
indices = np.arange(len(X))
dim_cell = Vol**(1/3) 
Diss = Diss_den * Vol
Trad = (Rad_den*prel.en_den_converter/prel.alpha_cgs)**(1/4)
R_vec = np.transpose(np.array([X, Y, Z]))
Rsph = np.linalg.norm(R_vec, axis = 1)
vel_vec = np.transpose(np.array([vx, vy, vz]))
vel = np.linalg.norm(vel_vec, axis=1)

orb_en = orb.orbital_energy(Rsph, vel, mass, params, prel.G) 
orb_en_spec = orb_en/mass
bern = orb.bern_coeff(Rsph, vel, den, mass, Press, IE_den, Rad_den, params)
long = np.arctan2(Y, X)          # Azimuthal angle in radians
lat = np.arccos(Z / Rsph)
v_rad, _, _ = to_spherical_components(vx, vy, vz, X, Y, Z)
ecc2 = orb.eccentricity_squared(R_vec, vel_vec, orb_en_spec, Mbh, G=prel.G)
ecc = np.array(np.sqrt(ecc2))
R_edge_wind = v_esc / prel.tsol_cgs * (tfb * t_fb_days_cgs)
xcr_w, ycr_w, cr_w = orb.make_cfr(R_edge_wind/apo)
wind_cond = np.logical_and(v_rad >= 0, bern > 0)


In [ ]:
x_line = np.arange(-40*Rt, 40*Rt, dtype=complex)
line_xz = draw_line(x_line, np.arcsin(2/3), 'line')
line_xz_neg = draw_line(x_line, np.pi-np.arcsin(2/3), 'line')
sections = choose_sections(X, Y, Z, choice = 'left_right_in_out_z')
cond_sec = []
label_obs = []
color_obs = []
fig, (ax_mid, ax_xz) = plt.subplots(1, 2, figsize = (18,8))
# indices = np.arange(len(X_wind))
for i, key in enumerate(sections.keys()):
    cond_sec.append(sections[key]['cond'])
    label_obs.append(sections[key]['label'])
    color_obs.append(sections[key]['color'])
    X_cond = X[sections[key]['cond']]
    Y_cond = Y[sections[key]['cond']]
    Z_cond = Z[sections[key]['cond']]  
    den_cond = den[sections[key]['cond']]     
    dim_cell_cond = dim_cell[sections[key]['cond']]
    cond_mid = np.logical_and(sections[key]['cond'], np.logical_and(np.abs(Z) < dim_cell, wind_cond))
    cond_y0 = np.logical_and(sections[key]['cond'], np.logical_and(np.abs(Y) < dim_cell, wind_cond))
    ax_mid.scatter(X[cond_mid]/Rt, Y[cond_mid]/Rt,  c = color_obs[i], s = 1, label = label_obs[i])
    ax_xz.scatter(X[cond_y0]/Rt, Z[cond_y0]/Rt,  c = color_obs[i], s = 1, label = label_obs[i])    

ax_xz.plot(x_line, line_xz, c = 'k', ls = 'dashed')
ax_xz.plot(x_line, line_xz_neg, c = 'k', ls = 'dashed')
ax_mid.set_xlabel(r'x [$r_{\rm t}$]')
ax_mid.set_ylabel(r'y [$r_{\rm t}$]')
ax_xz.set_xlabel(r'x [$r_{\rm t}$]')
ax_xz.set_ylabel(r'z [$r_{\rm t}$]')
ax_xz.set_xlim(-50, 20)
ax_xz.set_ylim(-25, 25)
# ax_mid.contour(xcfr05mb/Rt, ycfr05mb/Rt, cfr05mb/Rt, levels=[0], colors='k')
ax_mid.set_xlim(-15, 15)
ax_mid.set_ylim(-15, 15)
ax_xz.legend(fontsize = 16)
ax_xz.set_title(r'Y=0', fontsize = 20)
ax_mid.set_title('Mid-plane section', fontsize = 20)  

In [ ]:
out_bound_cond = np.logical_and(v_rad >= 0, bern < 0)
x_line = np.arange(-40*Rt, 40*Rt, dtype=complex)
line_xz = draw_line(x_line, np.arcsin(2/3), 'line')
line_xz_neg = draw_line(x_line, np.pi-np.arcsin(2/3), 'line')
sections = choose_sections(X, Y, Z, choice = 'left_right_in_out_z')

fig, (ax_mid, ax_xz) = plt.subplots(1, 2, figsize = (18,8))
cond_mid = np.logical_and(np.abs(Z) < dim_cell, wind_cond)
max_den = np.max(den[cond_mid])
ax_mid.scatter(X[cond_mid]/Rt, Y[cond_mid]/Rt,  c = 'dodgerblue', s = 1, label = r'$v_r\geq 0, \mathcal{B} \ge 0$', alpha = den[cond_mid]/max_den)

cond_y0 =np.logical_and(np.abs(Y) < dim_cell, wind_cond)
max_den = np.max(den[cond_y0])
ax_xz.scatter(X[cond_y0]/Rt, Z[cond_y0]/Rt,  c = 'dodgerblue', s = 1, label = r'$v_r\geq 0, \mathcal{B} \ge 0$', alpha = den[cond_y0]/max_den)  
 
cond_mid = np.logical_and(np.abs(Z) < dim_cell, out_bound_cond)
max_den = np.max(den[cond_mid])
ax_mid.scatter(X[cond_mid]/Rt, Y[cond_mid]/Rt,  c = 'firebrick', s = 1, label = r'$v_r\geq 0, \mathcal{B} < 0$', alpha = den[cond_mid]/max_den)

cond_y0 =np.logical_and(np.abs(Y) < dim_cell, out_bound_cond)
max_den = np.max(den[cond_y0])
ax_xz.scatter(X[cond_y0]/Rt, Z[cond_y0]/Rt,  c = 'firebrick', s = 1, label = r'$v_r\geq 0, \mathcal{B} < 0$', alpha = den[cond_y0]/max_den) 

ax_xz.plot(x_line, line_xz, c = 'k', ls = 'dashed')
ax_xz.plot(x_line, line_xz_neg, c = 'k', ls = 'dashed')
ax_mid.set_xlabel(r'x [$r_{\rm t}$]')
ax_mid.set_ylabel(r'y [$r_{\rm t}$]')
ax_xz.set_xlabel(r'x [$r_{\rm t}$]')
ax_xz.set_ylabel(r'z [$r_{\rm t}$]')
ax_xz.set_xlim(-50, 20)
ax_xz.set_ylim(-25, 25)
# ax_mid.contour(xcfr05mb/Rt, ycfr05mb/Rt, cfr05mb/Rt, levels=[0], colors='k')
ax_mid.set_xlim(-15, 15)
ax_mid.set_ylim(-15, 15)
ax_mid.legend(fontsize = 16)
ax_xz.set_title(r'Y=0', fontsize = 20)
ax_mid.set_title('Mid-plane section', fontsize = 20) 
plt.suptitle(f't = {tfb:.2f}' + r'$t_{\rm fb}$', fontsize = 20)
plt.savefig(f'{abspath}/Figs/{folder}/Wind/outflowSec_{snap}_alpha.png', dpi = 300)

In [ ]:

X_wind, Y_wind, Z_wind, mass_wind, den_wind, dim_cell_wind, Mass_wind, Press_wind, Temp_wind, vx_wind, vy_wind, vz_wind, IE_den_wind, Diss_den_wind, Rad_den_wind = \
make_slices([X, Y, Z, mass, den, dim_cell, Mass, Press, Temp, vx, vy, vz, IE_den, Diss_den, Rad_den], wind_cond)

In [ ]:
from Utilities.operators import choose_sections
sections = choose_sections(X_wind, Y_wind, Z_wind, choice = 'left_right_in_out_z')
cond_sec_wind = []
fig, ax = plt.subplots(1, 3, figsize = (24,8))
fig_mid, ax_mid = plt.subplots(1, 3, figsize = (24,8))
# indices = np.arange(len(X_wind))
for i, key in enumerate(sections.keys()):
    if key in ['south']:
        continue
    cond_sec_wind.append(sections[key]['cond'])
    X_cond = X_wind[sections[key]['cond']]
    Y_cond = Y_wind[sections[key]['cond']]
    Z_cond = Z_wind[sections[key]['cond']]  
    den_cond = den_wind[sections[key]['cond']]     
    dim_cell_cond = dim_cell_wind[sections[key]['cond']]
    cond_mid = np.logical_and(sections[key]['cond'], np.abs(Z_wind) < dim_cell_wind)
    img = ax[i].scatter(X_cond/Rt, Y_cond/Rt,  c = den_cond*prel.den_converter, s = .1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e-14, vmax = 1.5e-9))
    img_mid = ax_mid[i].scatter(X_wind[cond_mid]/Rt, Y_wind[cond_mid]/Rt,  c = den_wind[cond_mid]*prel.den_converter, s = .1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e-14, vmax = 1.5e-9))
    ax[i].contour(xcfr05mb/Rt, ycfr05mb/Rt, cfr05mb/Rt, levels=[0], colors='k')
    ax[i].set_xlabel(r'x [$r_{\rm t}$]')
    ax[i].set_xlim(-15, 15)
    ax[i].set_ylim(-15, 15)
    ax_mid[i].contour(xcfr05mb/Rt, ycfr05mb/Rt, cfr05mb/Rt, levels=[0], colors='k')
    ax_mid[i].set_xlabel(r'x [$r_{\rm t}$]')
    ax_mid[i].set_xlim(-15, 15)
    ax_mid[i].set_ylim(-15, 15)
ax[0].set_ylabel(r'y [$r_{\rm t}$]')
cbar = plt.colorbar(img)
cbar.set_label('Density [g cm$^{-3}$]')
cbar_mid = plt.colorbar(img_mid)
cbar_mid.set_label('Density [g cm$^{-3}$]')
fig_mid.suptitle('Mid-plane section')


In [ ]:
# curl_vec = np.load(f'{path}/curl_{snap}.npy')
# Curl magnitude
# curl_magnitude = np.linalg.norm(curl_vec, axis=1)

In [ ]:
z_chosen = 0
midplane = np.abs(Z-z_chosen) < dim_cell
X_midplane, Y_midplane, Z_midplane, dim_midplane, Mass_midplane, Den_midplane, Temp_midplane, Press_midplane, Diss_den_midplane, Rad_den_midplane, vel_midplane, vx_mid, vy_mid, orb_en_spec_mid = \
    sec.make_slices([X, Y, Z, dim_cell, mass, den, Temp, Press, Diss_den, Rad_den, vel, vx, vy, orb_en_spec], midplane)
vol_mid = dim_midplane**3
# curl_magnitude_mid = curl_magnitude[midplane]
Diss_den_midplane_cgs = Diss_den_midplane * prel.en_den_converter
Diss_midplane = Diss_den_midplane * vol_mid
# X_midplane, Y_midplane, Z_midplane, dim_midplane, Den_midplane, Temp_midplane, IE_den_midplane, Rad_den_midplane, VX_midplane, VY_midplane, VZ_midplane, Diss_den_midplane, IE_den_midplane, Press_midplane =\
# np.load(f'{abspath}/data/{folder}/slices/z/z0slice_{snap}.npy')
Trad_midplane = (prel.en_den_converter * Rad_den_midplane / prel.alpha_cgs)**(1/4)
cs_midplane = np.sqrt(gamma * Press_midplane / Den_midplane)
Den_midplane_cgs = Den_midplane * prel.den_converter
Diss_midplane_cgs = Diss_den_midplane * dim_midplane**3 * prel.en_converter
Rsph_mid = np.sqrt(X_midplane**2 + Y_midplane**2 + Z_midplane**2) 
long_mid = np.arctan2(Y_midplane, X_midplane)          # Azimuthal angle in radians
lat_mid = np.arccos(Z_midplane/ Rsph_mid)            # Elevation angle in radians
ratio_regime_mid = prel.alpha_cgs/prel.en_den_converter * Temp_midplane**4 / Press_midplane # Ratio of radiation pressure to gas pressure

# diss at pericenter
Rp_vec = np.array([[Rp, 0, 0]])
tree = KDTree(np.transpose(np.array([X_midplane, Y_midplane, Z_midplane])))
_, idx = tree.query(Rp_vec)
X_peri, Y_peri, Z_peri = X_midplane[idx], Y_midplane[idx], Z_midplane[idx]

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (12,8))
img = ax1.scatter(X_wind[np.abs(Z_wind)<dim_cell_wind]/Rt, Y_wind[np.abs(Z_wind)<dim_cell_wind]/Rt, c = den_wind[np.abs(Z_wind)<dim_cell_wind]*prel.den_converter
                  , s = .1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e-14, vmax = 1.5e-9))
cbar = plt.colorbar(img)
cbar.set_label(r'Den [g/cm$^3$]')
ax1.contour(xcfr05mb/Rt, ycfr05mb/Rt, cfr05mb/Rt, levels=[0], colors='k')
ax1.set_xlim(-20, 10)
ax1.set_ylim(-10, 10)
ax1.set_xlabel(r'X [$r_{t}$]')
ax1.set_ylabel(r'Y [$r_{t}$]')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (24,8))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane*prel.den_converter
                  , s = .1, cmap = 'plasma', norm = colors.LogNorm(vmin = 1e-10, vmax = 1e-4))
cbar = plt.colorbar(img)
cbar.set_label(r'Den [g/cm$^3$]')
ax1.set_xlim(-4, 1.5)
ax1.set_ylim(-1, 1)
ax1.set_xlabel(r'X [$R_{a}$]')
ax1.set_ylabel(r'Y [$R_{a}$]')
img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = Diss_den_midplane*prel.en_den_converter
                  , s = .1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e14/prel.Rsol_cgs, vmax = 1e19/prel.Rsol_cgs))
cbar = plt.colorbar(img)
cbar.set_label(r'Den diss [g/ s cm$^3$]')
ax2.set_xlim(-1, 0.5)
ax2.set_ylim(-.4, .4)
ax2.set_xlabel(r'X [$R_{a}$]')
plt.suptitle(r't/t$_{fb}$ = ' + f'{np.round(tfb,2)}, {check}', fontsize = 20)
plt.tight_layout()
# fig.savefig(f'{abspath}/Figs/next_meeting/{check}_slicez0_{snap}.png')

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize = (12,8))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = np.abs(orb_en_spec_mid)/E_mb
                  , s = .1, cmap = 'plasma', norm = colors.LogNorm(vmin = 1e-2, vmax = 10))
cbar = plt.colorbar(img)
cbar.set_label(r'$|$Specific OE$| [\Delta$E]')
ax1.set_xlim(-1.5, 0.2)
ax1.set_ylim(-.5, .5)
ax1.set_xlabel(r'X [$R_{a}$]')
plt.suptitle(r't/t$_{fb}$ = ' + f'{np.round(tfb,2)}, {check}', fontsize = 20)
plt.tight_layout()

In [ ]:
arrow_mask = np.logical_and(np.abs(X_midplane) < 1, np.abs(Y_midplane) < 1)
fig, ((axd, axDiss, axCs), (axDiv, axC, axT)) = plt.subplots(2,3, figsize = (20,20))
img = axd.scatter(X_midplane, Y_midplane, c = Den_midplane_cgs, cmap = 'plasma', s= .1, \
            norm = colors.LogNorm(vmin = 1e-3, vmax = 5e-1))
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'Density [g/cm$^3$]')
cb.ax.tick_params(which='major', length=7, width=1.2)
cb.ax.tick_params(which='minor', length=4, width=1)
axd.set_ylabel(r'$Y/R_\odot$')
img = axC.scatter(X_midplane, Y_midplane, c = curl_magnitude_mid, cmap = 'rainbow', s= .1, \
            norm = colors.LogNorm(vmin = 5e-2, vmax = 10))
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'$|\nabla \times \mathbf{v}|$')
cb.ax.tick_params(which='major', length=7, width=1.2)
cb.ax.tick_params(which='minor', length=4, width=1)

img = axDiss.scatter(X_midplane, Y_midplane, c = Diss_midplane_cgs, cmap = 'viridis', s= .1, \
            norm = colors.LogNorm(vmin = 1e38, vmax = 8e40))
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'Dissipation [erg/s]')
cb.ax.tick_params(which='major', length=7, width=1.2)
cb.ax.tick_params(which='minor', length=4, width=1)

img = axDiv.scatter(X_midplane, Y_midplane, c = np.abs(DivV_mid), cmap = 'coolwarm', s= .1, \
            norm = colors.LogNorm(vmin = 1e-2, vmax = 1e2))
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'$|\nabla \cdot \mathbf{v}|$')

img = axCs.scatter(X_midplane, Y_midplane, c = cs_midplane/vel_midplane, cmap = 'coolwarm', s= .1, \
            norm = colors.LogNorm(vmin = 1e-2, vmax = 1e2))
axCs.quiver(X_midplane[arrow_mask][::200], Y_midplane[arrow_mask][::200], vx_mid[arrow_mask][::200], vy_mid[arrow_mask][::200], angles='xy', scale_units='xy', scale=1, color="k", width=0.003, headwidth = 4)
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'$c_{\rm s}/v$')
img = axT.scatter(X_midplane, Y_midplane, c = Trad_midplane, cmap = 'rainbow', s= .1, \
            norm = colors.LogNorm(vmin = 1e4, vmax = 1e7))
cb = plt.colorbar(img, orientation='horizontal')
cb.set_label(r'$T_{\rm rad}$ [K]')
fig.suptitle(f't = {np.round(tfb, 4)}' + r'$t_{\rm fb}$, z = ' + str(z_chosen), fontsize = 24)
fig.tight_layout()
for ax in [axd, axC, axDiss, axCs, axDiv, axT]:
    if ax not in [axDiv, axT, axC]:
        ax.set_xlabel(r'$X [R_\odot]$')
    ax.set_xlim(-5, 5)#(-340,25)
    ax.set_ylim(-5, 5)#(-70,70)
    if ax in [axd, axDiss, axDiv]:
        ax.set_ylabel(r'$Y [R_\odot]$')
    ax.set_aspect('equal', 'box')

# fig.savefig(f'{abspath}/Figs/{folder}/slices/{snap}Shock_z{z_chosen}.png')

In [ ]:
# en_stream = np.load(f'{abspath}/data/{folder}/WH/enclosed/indeces_enclosed_{check}{snap}.npy', allow_pickle=True)
# all_stream = np.concatenate(en_stream[187:190])
# theta_arr, x_stream, y_stream, z_stream, _ = \
#         np.load(f'/Users/paolamartire/shocks/data/{folder}/WH/stream/stream_{check}{snap}.npy' )

xph, yph, zph, volph, denph, Tempph, Rad_denph, Vxph, Vyph, Vzph, Pressph, IE_denph, _, _, _, _ = \
     np.loadtxt(f'/Users/paolamartire/shocks/data/{folder}/photo/{check}_photo{snap}.txt')

data_tr= np.load(f'{abspath}/data/{folder}/trap/{check}_Rtr{snap}.npz')
x_tr, y_tr, z_tr,  den_tr, vol_tr, v_tr, ie_den_tr, rad_den_tr, press_tr = \
    data_tr['x_tr'], data_tr['y_tr'], data_tr['z_tr'], data_tr['den_tr'], data_tr['vol_tr'], data_tr['Vr_tr'], data_tr['IE_den_tr'], data_tr['Rad_den_tr'], data_tr['P_tr']
den_tr/= prel.den_converter #beacuse it was in CGS
mass_tr = den_tr * vol_tr
r_tr_all = np.sqrt(x_tr**2 + y_tr**2 + z_tr**2)
bern_tr = orb.bern_coeff(r_tr_all, v_tr, den_tr, mass_tr, press_tr, ie_den_tr, rad_den_tr, params)
energy_tr = orb.orbital_energy(r_tr_all, v_tr, mass_tr, params, prel.G)

In [ ]:
bins_r_over_apo = np.arange(0,5,0.05)
bins_X_over_apo = np.arange(-5,5,0.05)
unbound = bern > 0 
mass_unb, Rsph_unb, X_unb = Mass[unbound], Rsph[unbound], X[unbound]
mass_r_binned, bins_r_edges = np.histogram(Rsph_unb/apo, bins = bins_r_over_apo, weights = mass_unb)
mass_X_binned, bins_X_edges = np.histogram(X_unb/apo, bins = bins_X_over_apo, weights = mass_unb)
# to reproduce some plots from Price+24
# mass_binned, bins_edges = np.histogram(Rsph/apo, bins = bins_r_over_apo, weights = Mass)
# # Unweighted mean density per bin
# counts, _   = np.histogram(Rsph_unb/apo, bins=bins_edges)
# sum_den, _  = np.histogram(Rsph_unb/apo, bins=bins_edges, weights=den_unb)
# mean_den = np.divide(
#     sum_den, counts,
#     out=np.full_like(sum_den, np.nan, dtype=float),
#     where=counts > 0
# )

bin_centers_r = 0.5 * (bins_r_edges[:-1] + bins_r_edges[1:])
bin_centers_X = 0.5 * (bins_X_edges[:-1] + bins_X_edges[1:])
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (16,6))
ax1.plot(bin_centers_r, mass_r_binned/mstar, c = 'k')
ax1.axhline(.5, c = 'b', ls = ':')
ax1.set_xlabel(r'$R/R_{\rm a}$')
ax1.set_ylabel(r'Unbound mass [$M_\star$]')
ax1.set_yscale('log')

ax2.plot(bin_centers_X, mass_X_binned/mstar, c = 'k')
ax2.axvline(-a_mb/apo, c = 'b', ls = '--')
ax2.axhline(.5, c = 'b', ls = ':')
ax2.set_xlabel(r'$X/R_{\rm a}$')
ax2.set_yscale('log')



In [ ]:
outflow = np.logical_and(bern>= 0, v_rad>0)
midplane = np.abs(Z) < dim_cell
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (25,8))
img = ax1.scatter(Rsph[np.logical_and(midplane, outflow)]/Rp, np.abs(Z[np.logical_and(midplane, outflow)])/Rp, c = Mass[np.logical_and(midplane, outflow)], s = 1, norm = colors.LogNorm(vmin = 1e-12, vmax = 1e-7), cmap = 'rainbow')
cbar = plt.colorbar(img)
cbar.set_label(r'Mass [$M_\odot$]')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
img = ax2.scatter(Rsph[np.logical_and(midplane, outflow)]/Rp, np.abs(Z[np.logical_and(midplane, outflow)])/Rp, c = v_rad[np.logical_and(midplane, outflow)]/v_esc, s = 1, norm = colors.LogNorm(vmin = 5e-2, vmax = 1), cmap = 'rainbow')
cbar = plt.colorbar(img)
cbar.set_label(r'$v_{\rm r} [v_{\rm esc}]$')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
img = ax3.scatter(Rsph[np.logical_and(midplane, outflow)]/Rp, np.abs(Z[np.logical_and(midplane, outflow)])/Rp, c = v_rad[np.logical_and(midplane, outflow)]*Mass[np.logical_and(midplane, outflow)], s = 1, norm = colors.LogNorm(vmin = 1e-11, vmax = 1e-7), cmap = 'rainbow')
cbar = plt.colorbar(img)
cbar.set_label(r'$v_{\rm r}\cdot M $')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
for ax in (ax1, ax2, ax3):
    ax.loglog()
    ax.set_xlim(1, 200)
    ax.set_ylim(1e-2, 200)
    ax.set_xlabel(r'$R [R_{\rm p}]$')
    ax.tick_params(axis = 'both', length = 9, width = 1.2)
    ax.tick_params(axis = 'both', which = 'minor', length = 7, width = .8)
ax1.set_ylabel(r'$|Z| [R_p]$')
plt.suptitle(f'Outflow material in the midplane at t = {np.round(tfb, 2)}' + r'$t_{\rm fb}$', fontsize = 20)
plt.tight_layout()


In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20,8))
img = ax1.scatter(Rsph[np.logical_and(np.abs(Z)<Vol**(1/3), outflow)][::5]/Rp, np.abs(Z[np.logical_and(np.abs(Z)<Vol**(1/3), outflow)][::5]), c = (Vol[np.logical_and(np.abs(Z)<Vol**(1/3), outflow)][::5])**(1/3), s = 1, norm = colors.LogNorm(vmin = 1e-1, vmax = 1e3), cmap = 'rainbow')
cbar = plt.colorbar(img)
cbar.set_label(r'dim cell [$R_\odot$]')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
# img = ax2.scatter(Rsph[outflow][::5]/Rp, np.abs(Z[outflow][::5]), c = v_rad[outflow][::5]/v_esc, s = 1, norm = colors.LogNorm(vmin = 5e-2, vmax = 1), cmap = 'rainbow')
# cbar = plt.colorbar(img)
# cbar.set_label(r'$v_{\rm r} [v_{\rm esc}]$')
# cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
for ax in (ax1, ax2):
    ax.loglog()
    ax.set_xlim(1e-1, 200)
    ax.set_ylim(1e-1, 1e4)
    ax.set_xlabel(r'$R [R_{\rm p}]$')
    ax.tick_params(axis = 'both', length = 9, width = 1.2)
    ax.tick_params(axis = 'both', which = 'minor', length = 7, width = .8)
ax1.set_ylabel(r'$|Z| [R_\odot]$')
plt.suptitle(f'Outflow material at t = {np.round(tfb, 2)}' + r'$t_{\rm fb}$', fontsize = 20)
plt.tight_layout()

In [ ]:
outflow_mid_B = np.logical_and(bern_mid>= 0, v_rad_mid>0)
outflow_mid_OE = np.logical_and(orb_en_mid>= 0, v_rad_mid>0) 
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (20,7))
img = ax1.scatter(X_midplane/Rp, Y_midplane/Rp, c = orb_en_mid/Mass_midplane, s = 1, cmap = 'coolwarm', vmin = -50, vmax = 50)
cbar = plt.colorbar(img, label = r'$\varepsilon$ [code units]')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
cbar.ax.tick_params(axis = 'both', which = 'minor', length = 7, width = .8)
img = ax2.scatter(X_midplane/Rp, Y_midplane/Rp, c = bern_mid, s = 1, vmin = -50, vmax = 50, cmap = 'coolwarm')
cbar = plt.colorbar(img, label = r'$\mathcal{B}$')
cbar.ax.tick_params(axis = 'both', length = 9, width = 1.2)
cbar.ax.tick_params(axis = 'both', which = 'minor', length = 7, width = .8)
 
for ax in (ax1, ax2):
    ax.set_xlim(-100, 50) 
    ax.set_ylim(-50, 50)
    ax.set_xlabel(r'X [$r_p$]')
ax1.set_ylabel(r'Y [$r_p$]')
plt.suptitle(r't/t$_{fb}$ = ' + f'{np.round(tfb,2)}' , fontsize = 20)
plt.tight_layout()
plt.savefig(f'{abspath}/Figs/section{check}{snap}_whereUnb.png', bbox_inches = 'tight')

In [ ]:
# Radiation pressure dominated?
fig, ax1 = plt.subplots(1,1, figsize = (9,5))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = ratio_regime_mid, s = 2, cmap = 'rainbow', norm = colors.LogNorm(vmin = 0.1, vmax = 10))
cbar = plt.colorbar(img)
cbar.set_label(r'$P_{\rm rad} / P_{\rm gas}$')
ax1.set_xlim(-5, 5)
ax1.set_ylim(-4, 4)
ax1.set_xlabel(r'X [$R_a$]')
ax1.set_ylabel(r'Y [$R_a$]')
plt.suptitle(r't/t$_{fb}$ = ' + f'{np.round(tfb,2)}, run: {check_name}', fontsize = 20)
plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (21,7), width_ratios=[1.5, 1])
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, 
            c = bern_mid/E_mb, vmin = -6, vmax = 10,
            cmap = 'jet', s = 2, label = r'B$ < 0$')    
cbar = plt.colorbar(img, label = r'$|\mathcal{B}/\Delta\epsilon|$')
ax1.scatter(x_stream/apo, y_stream/apo, c = 'k', s = 2, label = r'stream')
ax1.set_xlim(-4, 2)
ax1.set_ylim(-2,2)
ax1.set_xlabel(r'X [$R_a$]')
ax1.set_ylabel(r'Y [$R_a$]')

img = ax2.scatter(X_midplane/Rt, Y_midplane/Rt, 
            c = bern_mid/E_mb, vmin = -6, vmax = 0,
            cmap = 'viridis', s = 2, label = r'B$ < 0$')
cbar = plt.colorbar(img, label = r'$|\mathcal{B}/\Delta\epsilon|$')
ax2.scatter(x_stream/Rt, y_stream/Rt, c = 'k', s = 2, label = r'stream')
ax2.set_xlim(-1, 1)
ax2.set_ylim(-1,1)
ax2.set_xlabel(r'X [$R_t$]')
ax2.set_ylabel(r'Y [$R_t$]')
plt.suptitle(r't/t$_{fb}$ = ' + f'{np.round(tfb,2)}, run: {check_name}, ($\Delta\epsilon=${np.round(E_mb,2)})' , fontsize = 20)
plt.tight_layout()

In [ ]:
Diss_den_midplane_cgs = Diss_den_midplane * prel.en_den_converter/prel.tsol_cgs
Den_diss_peri_cgs = Den_diss_peri * prel.den_converter
print('Diss at pericenter: ', Den_diss_peri_cgs)
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (15,5))
img = ax1.scatter(X_midplane[Diss_den_midplane>0]/apo, Y_midplane[Diss_den_midplane>0]/apo, c = Diss_midplane_cgs[Diss_den_midplane>0], s = 1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e30, vmax = 1e38))
cbar = plt.colorbar(img)
cbar.set_label(r'$\dot{E}_{\rm diss}$ [erg/s]')
ax1.set_title('Positive diss')

# img = ax2.scatter(X_midplane[Diss_den_midplane<0]/apo, Y_midplane[Diss_den_midplane<0]/apo, c = np.abs(Diss_den_midplane_cgs[Diss_den_midplane<0]), s = 1, cmap = 'viridis', norm = colors.LogNorm(vmin = 1e-1, vmax = 1e6))
# cbar = plt.colorbar(img)
# cbar.set_label(r'$|\dot{E}_{\rm diss}|$ [erg/s/cm$^3$]')
# ax2.set_title('Negative diss')

# if Den_diss_peri_cgs> 0:
#     ax1.scatter(X_peri/apo, Y_peri/apo, c = Den_diss_peri_cgs, s = 10, edgecolors = 'r', facecolors = 'none', label = r'R$_p$')
# else: 
#     ax2.scatter(X_peri/apo, Y_peri/apo, c = np.abs(Den_diss_peri_cgs), s = 10, edgecolors = 'r', facecolors = 'none', label = r'R$_p$')

for ax in [ax1, ax2]:
    ax.set_xlim(-10, 5)
    ax.set_ylim(-2, 7)
    ax.set_xlabel(r'X [$R_a$]')
    ax.legend(fontsize=14)
plt.suptitle(r'Dissipation density rate at t/t$_{fb}$ = ' + f'{np.round(tfb,2)}, run: {check_name}', fontsize = 20)
plt.tight_layout()

YZ slice


In [ ]:
ychosen = 0
yz = np.abs(Y-ychosen) < dim_cell
X_yz, Y_yz, Z_yz, dim_yz, Mass_yz, Den_yz, Press_yz, Temp_yz, Diss_den_yz, v_rad_yz, VX_yz, VY_yz, VZ_yz, orb_en_yz, v_rad_yz, bern_yz = \
    sec.make_slices([X, Y, Z, dim_cell, mass, den, Press, Temp, Diss_den, v_rad, vx, vy, vz, orb_en, v_rad, bern], yz)


In [ ]:
wind_cond_yz = np.logical_and(v_rad_yz >= 0, bern_yz > 0)
# X_wind, Y_wind, Z_wind, mass_wind, den_wind, dim_cell_wind, Mass_wind, Press_wind, Temp_wind, vx_wind, vy_wind, vz_wind, IE_den_wind, Diss_den_wind, Rad_den_wind = \
# make_slices([X, Y, Z, mass, den, dim_cell, Mass, Press, Temp, vx, vy, vz, IE_den, Diss_den, Rad_den], wind_cond)

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (12, 8))
img = ax1.scatter(X_yz[wind_cond_yz]/Rt, Z_yz[wind_cond_yz]/Rt, c = Den_yz[wind_cond_yz]*prel.den_converter, s = 1, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e-13, vmax = 1e-8))
cbar = plt.colorbar(img)
ax1.scatter(X_yz[~wind_cond_yz]/Rt, Z_yz[~wind_cond_yz]/Rt, c = 'k', s = 1)
cbar.set_label(r'Density [$\mathrm{g\,cm^{-3}}$]')
ax1.set_xlabel(r'X [$R_{\rm t}$]')
ax1.set_ylabel(r'Z [$R_\odot$]')
ax1.set_xlim(-80, 80)
ax1.set_ylim(-80, 80)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (25,8))
img = ax1.scatter(Y_yz, Z_yz, c = Den_yz*prel.den_converter, s = 10, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e-10, vmax = 4e-6))
cbar = plt.colorbar(img)
cbar.set_label(r'Density [g/cm$^3$]')
ax1.set_ylabel(r'Z [$R_\odot$]')

# ax2.scatter(Y_yz[np.logical_and(orb_en_yz>0, v_rad_yz<0)]/amin, Z_yz[np.logical_and(orb_en_yz>0, v_rad_yz<0)]/amin, s = 10, c = 'orange', label = 'in unbound')
# ax2.scatter(Y_yz[np.logical_and(orb_en_yz<0, v_rad_yz<0)]/amin, Z_yz[np.logical_and(orb_en_yz<0, v_rad_yz<0)]/amin, s = 10, c = 'b', label = 'in bound')
# ax2.scatter(Y_yz[np.logical_and(orb_en_yz<0, v_rad_yz>0)]/amin, Z_yz[np.logical_and(orb_en_yz<0, v_rad_yz>0)]/amin, s = 10, c = 'dodgerblue', label = 'out bound')
# ax2.scatter(Y_yz[np.logical_and(orb_en_yz>=0, v_rad_yz>=0)]/amin, Z_yz[np.logical_and(orb_en_yz>=0, v_rad_yz>=0)]/amin, s = 10, c = 'r', label = 'out unbound')
# # ax2.quiver(Y_yz[::1000]/amin, Z_yz[::1000]/amin, VY_yz[::1000]/140, VZ_yz[::1000]/140, angles='xy', scale_units='xy', scale=0.5, color="k", width=0.003, headwidth = 6)
# img = ax3.scatter(Y_yz/amin, Z_yz/amin, c = np.abs(v_rad_yz), norm = colors.LogNorm(vmin = 1e-1, vmax = 4e1), s = 10, cmap = 'rainbow')
# cbar = plt.colorbar(img)
# cbar.set_label(r'$v_R$ [code units]')
for ax in [ax1, ax2, ax3]:
    ax.scatter(0,0,c= 'k', marker = 'x', s=80)
    ax.set_xlim(-Rp/2, Rp/2)
    ax.set_ylim(-Rp/2, Rp/2)
    ax.set_xlabel(r'Y [$R_\odot$]')
    ax.contour(xcfr_grid[2], ycfr_grid[2], cfr_grid[2], levels=[0], colors='k')
    # ax.axvline(0.5, color = 'k', linestyle = 'dotted')
    # ax.axhline(0.5, color = 'k', linestyle = 'dotted')
    # ax.axvline(-0.5, color = 'k', linestyle = 'dotted')
    # ax.axhline(-0.5, color = 'k', linestyle = 'dotted')
ax2.legend(fontsize = 20)   
plt.suptitle(r'Cross section, X=0, t/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()
# plt.savefig(f'{saving_path}/slices/yz{threshx}_{snap}.png')

In [ ]:
v_rad_05amin = v_rad[np.abs(Rsph-0.5*amin)<dim_cell]
rhov_05amin = den[np.abs(Rsph-0.5*amin)<dim_cell] * v_rad_05amin
plt.plot(np.abs(rhov_05amin[v_rad_05amin<0]), label = 'in')
plt.plot(rhov_05amin[v_rad_05amin>=0], label = 'out')
plt.legend()
plt.yscale('log')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (18,5))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane, s = 2, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e-11, vmax = 1e-6))
cbar = plt.colorbar(img)
cbar.set_label(r'Density $[M_\odot]$')
ax1.set_ylabel(r'Y [$R_a$]')

# img = ax2.scatter(X_midplane/amin, Y_midplane/amin, s = 2, c = Diss_den_midplane*prel.en_den_converter/prel.tsol_cgs, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e2, vmax = 1e8))
img = ax2.scatter(X_midplane/amin, Y_midplane/amin, s = 2, c = Temp_midplane, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e3, vmax = 1e6))
cbar = plt.colorbar(img)
cbar.set_label(r'T [K]')
# cbar.set_label(r'Diss density rate [erg/s/cm$^3$]')
# ax2.quiver(X_midplane[::1000]/amin, Y_midplane[::1000]/amin, VY_mid[::1000]/140, VZ_mid[::1000]/140, angles='xy', scale_units='xy', scale=0.5, color="k", width=0.003, headwidth = 6)
for ax in [ax1, ax2]:
    ax.set_xlim(-1.2, 1)
    ax.set_ylim(-.5, .5)
    ax.set_xlabel(r'X [$R_a$]')
    # ax.contour(xcfr_grid[2], ycfr_grid[2], cfr_grid[2], levels=[0], colors='k')
    # ax.axvline(0.5, color = 'k', linestyle = 'dotted')
    # ax.axhline(0.5, color = 'k', linestyle = 'dotted')
    # ax.axvline(-0.5, color = 'k', linestyle = 'dotted')
    # ax.axhline(-0.5, color = 'k', linestyle = 'dotted')
plt.suptitle(r'Cross section, Z=0, t/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()


In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (25,8))
img = ax1.scatter(X_midplane/amin, Y_midplane/amin, c = Den_midplane, s = 10, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e-11, vmax = 1e-6))
cbar = plt.colorbar(img)
cbar.set_label(r'Density $[M_\odot]$')
ax1.set_ylabel(r'Y [$a_{mb}$]')

ax2.scatter(X_midplane[np.logical_and(orb_en_mid>0, v_rad_mid<0)]/amin, Y_midplane[np.logical_and(orb_en_mid>0, v_rad_mid<0)]/amin, s = 10, c = 'orange', label = 'in unbound')
ax2.scatter(X_midplane[np.logical_and(orb_en_mid<0, v_rad_mid<0)]/amin, Y_midplane[np.logical_and(orb_en_mid<0, v_rad_mid<0)]/amin, s = 10, c = 'b', label = 'in bound')
ax2.scatter(X_midplane[np.logical_and(orb_en_mid<0, v_rad_mid>0)]/amin, Y_midplane[np.logical_and(orb_en_mid<0, v_rad_mid>0)]/amin, s = 10, c = 'dodgerblue', label = 'out bound')
ax2.scatter(X_midplane[np.logical_and(orb_en_mid>=0, v_rad_mid>=0)]/amin, Y_midplane[np.logical_and(orb_en_mid>=0, v_rad_mid>=0)]/amin, s = 10, c = 'r', label = 'out unbound')

img = ax3.scatter(X_midplane/amin, Y_midplane/amin, c = np.abs(v_rad_mid), norm = colors.LogNorm(vmin = 1e-1, vmax = 3e1), s = 10, cmap = 'rainbow')
cbar = plt.colorbar(img)
cbar.set_label(r'$v_R$ [code units]')
# ax2.quiver(X_midplane[::1000]/amin, Y_midplane[::1000]/amin, VY_mid[::1000]/140, VZ_mid[::1000]/140, angles='xy', scale_units='xy', scale=0.5, color="k", width=0.003, headwidth = 6)
for ax in [ax1, ax2, ax3]:
    ax.set_xlim(-10, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel(r'X [$a_{mb}$]')
    ax.contour(xcfr_grid[2], ycfr_grid[2], cfr_grid[2], levels=[0], colors='k')
    ax.axvline(0.5, color = 'k', linestyle = 'dotted')
    ax.axhline(0.5, color = 'k', linestyle = 'dotted')
    ax.axvline(-0.5, color = 'k', linestyle = 'dotted')
    ax.axhline(-0.5, color = 'k', linestyle = 'dotted')
ax2.legend(fontsize = 20)   
plt.suptitle(r'Cross section, Z=0, t/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()


In [ ]:
# are we pressure dominated?
RadP_P = np.abs(prel.alpha_cgs/(3*prel.en_den_converter) * Temp**4 / Press)
RadP_P = np.abs(prel.alpha_cgs/(3*prel.en_den_converter) * Temp**4 / Press)
RadP_Pmid = np.abs(prel.alpha_cgs/(3*prel.en_den_converter) * Temp_midplane**4 / Press_midplane)
plt.scatter(X_midplane/apo, Y_midplane/apo, c = RadP_Pmid, s = 5, cmap = 'rainbow', norm = colors.LogNorm(vmin = 1e-2, vmax = 1e2))
plt.colorbar(label = r'$\frac{P_{\rm rad}}{P_{\rm gas}}$')
plt.xlabel(r'X [$R_{a}$]')
plt.ylabel(r'Y [$R_{a}$]')
plt.xlim(-7,3)
plt.ylim(-4,4)
plt.title(f'Total radiation dominated cells: {int(len(RadP_P[RadP_P>=1])/len(RadP_P)*100)}\% ' + r'at t/t$_{\rm fb}$ = ' + str(np.round(tfb,2)), fontsize = 18)
plt.savefig(f'{saving_path}pressure_dom{snap}.png')


In [ ]:
theta_prec = orb.precession_angle(Rstar, mstar, Mbh, beta, prel.csol_cgs, prel.G)
R_si = orb.R_selfinter(Rstar, mstar, Mbh, beta, prel.csol_cgs, prel.G)
print(f'Precession angle: {theta_prec} rad, {theta_prec*180/np.pi} degrees')
print(f'Self-intersection radius: {R_si} Rsol')


In [ ]:
import matlab.engine
from src.Opacity.linextrapolator import nouveau_rich
eng = matlab.engine.start_matlab()
opac_path = f'{abspath}/src/Opacity'
T_cool = np.loadtxt(f'{opac_path}/T.txt')
Rho_cool = np.loadtxt(f'{opac_path}/rho.txt')
rossland = np.loadtxt(f'{opac_path}/ross.txt')
T_cool2, Rho_cool2, rossland2 = nouveau_rich(T_cool, Rho_cool, rossland, what = 'scattering', slope_length = 5)
sigma_rossland = eng.interp2(T_cool2, Rho_cool2, rossland2.T, np.log(Temp_midplane), np.log(Den_midplane_cgs), 'linear', 0)
sigma_rossland = np.array(sigma_rossland)[0]
sigma_rossland_eval = np.exp(sigma_rossland) # [1/cm]
sigma_rossland_eval[sigma_rossland == 0.0] = 1e-20
tau_mid = sigma_rossland_eval * dim_midplane * prel.Rsol_cgs 
scatt_mid = 0.34*Den_midplane*prel.den_converter * dim_midplane * prel.Rsol_cgs

fig, ax1 = plt.subplots(1,1, figsize = (8,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Mass_midplane, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 1e-12, vmax = 1e-9))
cbar = plt.colorbar(img)
cbar.set_label(r'Mass', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)
ax1.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.title(f'{folder} ' + r't/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()

fig, ax2 = plt.subplots(1,1, figsize = (8,6))
img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = tau_mid, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 9e-1, vmax = 1e4))
cbar = plt.colorbar(img)
cbar.set_label(r'$\tau$', fontsize = 16)
for ax in [ax1, ax2]:
    ax.axhline(2*Rt/apo, c = 'k')
    ax.axhline(-2*Rt/apo, c = 'k')
    ax.axvline(2*Rt/apo, c = 'k')
    ax.axvline(-2*Rt/apo, c = 'k')
    ax.set_xlim(-.2,.2)
    ax.set_ylim(-.2, .2)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.title(f'{folder} ' + r't/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()
    

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (8,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Diss_den_midplane, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 5e-10, vmax = 1e-7))
cbar = plt.colorbar(img)
cbar.set_label(r'$\rho$ [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)
ax1.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.title(f'{folder} ' + r't/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()

In [ ]:
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2,3, figsize = (25,12))
img = ax1.scatter(X_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, Y_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, c = Den_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]*prel.den_converter, s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin =1e-8, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\rho$ [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)

img = ax2.scatter(X_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, Y_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, c = Mass_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)], s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin = 2e-6, vmax = 5e-4))
cbar = plt.colorbar(img)
cbar.set_label(r'mass [M$_\odot$]', fontsize = 16)

img = ax3.scatter(X_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, Y_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)]/apo, c = dim_midplane[np.logical_and(orb_en_spec_mid>0.5*delta_E, orb_en_spec_mid<0.6*delta_E)], s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin = 1, vmax = 20))
cbar = plt.colorbar(img)
cbar.set_label(r'R [R$_\odot$]', fontsize = 16)

for ax in [ax1, ax2, ax3]:
    ax.set_xlim(-6, -2)
    ax.set_ylim(-3, 0)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)

img = ax4.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane*prel.den_converter, s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin =1e-8, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\rho$ [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)

img = ax5.scatter(X_midplane/apo, Y_midplane/apo, c = Mass_midplane, s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin=2e-10, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'mass [M$_\odot$]', fontsize = 16)

img = ax6.scatter(X_midplane/apo, Y_midplane/apo, c = dim_midplane, s = 2, cmap = 'turbo', norm = colors.LogNorm(vmin = 4e-2, vmax = 2))
cbar = plt.colorbar(img)
cbar.set_label(r'R [R$_\odot$]', fontsize = 16)

for ax in [ax4, ax5, ax6]:
    ax.set_xlim(-.2, .2)
    ax.set_ylim(-.2, .2)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)

plt.suptitle(f'tot cells in the box {int(len(data.X)*1e-6)}' + r'$\cdot10^6$' + f', {check}', fontsize = 20)
plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (18,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane*prel.den_converter, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin =1e-8, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\rho$ [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)

img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = kappa_mid, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 1e-2, vmax=1e5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\kappa$', fontsize = 16)
# img = ax2.scatter(X_midplane[kappa_mid<20]/apo, Y_midplane[kappa_mid<20]/apo, c = 'b', s = 1)

for ax in [ax1, ax2]:
    # ax.scatter(observers_xyz[90:94,0], observers_xyz[90:94,1], c = 'k', s=20, label = 'observers')
    ax.scatter(0,0,c= 'k', marker = 'x', s=80)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.suptitle(r'Orbital plane, t/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)
plt.tight_layout()
plt.savefig(f'{saving_path}kappa_{snap}.png')


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (18,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane*prel.den_converter, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin =1e-8, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\rho$ [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)

img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = kappa_mid, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 1e-2, vmax=1e5))
cbar = plt.colorbar(img)
cbar.set_label(r'$\kappa$', fontsize = 16)
img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = Temp_midplane, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin = 1e3, vmax=1e6))
cbar = plt.colorbar(img)

for ax in [ax1, ax2]:
    ax.scatter(observers_xyz[90:94,0], observers_xyz[90:94,1], c = 'k', s=20, label = 'observers')
    ax.scatter(0,0,c= 'k', marker = 'x', s=80)
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)
ax2.legend()
plt.suptitle(r'Orbital plane, t/t$_{fb}$ = ' + str(np.round(tfb,3)))
plt.tight_layout()
# plt.savefig(f'{saving_path}kappa_{snap}.png')

In [ ]:
fig, ax1 = plt.subplots(1,1, figsize = (14,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Den_midplane*prel.den_converter, s = 1, cmap = 'turbo', norm = colors.LogNorm(vmin =1e-12, vmax = 1e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'Density [g/cm$^3$]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)
ax1.set_xlim(-4,.5)
ax1.set_ylim(-.5, .5)
ax1.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.title(f'{folder} ' + r't/t$_{fb}$ = ' + str(np.round(tfb,3)), fontsize = 20)

In [ ]:
idxRp_lines_mid = np.concatenate(np.where(np.logical_and(np.abs(Y_midplane)<dim_midplane, np.abs(X_midplane-Rp)<dim_midplane))) #X=Rp, Y=0
PressRp_mid, DenRp_mid, XRp_mid, YRp_mid, dimRp_mid, TRp_mid = Press_midplane[idxRp_lines_mid], Den_midplane[idxRp_lines_mid], X_midplane[idxRp_lines_mid], Y_midplane[idxRp_lines_mid], dim_midplane[idxRp_lines_mid], Temp_midplane[idxRp_lines_mid]
idxRp_mid = np.argmin(np.abs(XRp_mid-Rp)) #Z=0, Y=0, X=Rp
print('Rp coordinates X, Y: ', XRp_mid[idxRp_mid], YRp_mid[idxRp_mid])
csRp_mid = np.sqrt(PressRp_mid[idxRp_mid]/DenRp_mid[idxRp_mid])
Hrp_mid = csRp_mid/omegaRp
print(f'Minimum R cell size: {np.min(dim_midplane)}')
print(f'T = {TRp_mid[idxRp_mid]/1e6} 1e6 K')
print(f'Cs at Rp: {csRp_mid*prel.Rsol_cgs*1e-7/prel.tsol_cgs} 1e-7 cm/s \nH at Rp: {Hrp_mid} Rsol')
print(f'max numbers of cell in H: {Hrp_mid/np.min(dim_midplane)}')

In [ ]:
absvel = True
fig, ax = plt.subplots(1,1, figsize = (12,6))
if absvel:
    img = ax.scatter(X_midplane/apo, Y_midplane/apo, c = np.abs(v_rad_mid_kms), s = 4, cmap = 'jet', norm = colors.LogNorm(vmin = 5e2, vmax = 2e4))
else:
    img = ax.scatter(X_midplane/apo, Y_midplane/apo, c = v_rad_mid_kms*1e-4, s = 4, cmap = 'jet', vmin = -1, vmax = 1)#, vmin = 5e2, vmax = 2e4)
cbar = plt.colorbar(img)
ax.scatter(xph[indecesorbital]/apo, yph[indecesorbital]/apo, facecolor = 'none', edgecolors = 'k')
ax.scatter(0,0,c= 'k', marker = 'x', s=80)
ax.set_xlim(-6, 2.5)
ax.set_ylim(-3, 2)
ax.set_xlabel(r'X [$R_a$]', fontsize = 18)
ax.set_ylabel(r'Y [$R_a$]', fontsize = 18)
plt.title(r'Cross section, $|z|<V^{1/3}$, t/t$_{fb}$ = ' + str(np.round(tfb,3)))
if absvel:
    cbar.set_label(r'$|$Radial velocity$|$ [km/s]', fontsize = 16)
    plt.tight_layout()
    plt.savefig(f'{saving_path}/slices/absvel{snap}.png')
else:
    cbar.set_label(r'Radial velocity [10$^4$ km/s]', fontsize = 16)
    plt.tight_layout()
    plt.savefig(f'{saving_path}/slices/vel{snap}.png')


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (15,6))
img = ax1.scatter(X_midplane/apo, Y_midplane/apo, c = Diss_midplane, s = 1, cmap = 'inferno', vmin = -1e-15, vmax = 1e-15)
cbar = plt.colorbar(img)
cbar.set_label(r'Dissipation rate [energy/time]', fontsize = 16)
ax1.set_ylabel(r'Y [$R_a$]', fontsize = 18)

img = ax2.scatter(X_midplane/apo, Y_midplane/apo, c = divV_mid, s = 1, cmap = 'inferno', vmin = -1, vmax=1)#1e2, vmax = 1e9)
cbar = plt.colorbar(img)
cbar.set_label(r'$\nabla v$ [1/time]', fontsize = 16)
for ax in [ax1, ax2]:
    ax.scatter(0,0,c= 'k', marker = 'x', s=80)
    ax.set_xlim(-1.2, 0.3)
    ax.set_ylim(-0.5, 0.5)
    ax.set_xlabel(r'X [$R_a$]', fontsize = 18)
plt.suptitle(r'Orbital plane, t/t$_{fb}$ = ' + str(np.round(tfb,3)))
plt.tight_layout()
plt.savefig(f'{saving_path}midplaneDiss_{snap}.png')

In [ ]:
# fig, ax = plt.subplots(1,1, figsize = (12,4))
# img = ax.scatter(X_midplane/apo, Y_midplane/apo, c = EdotDiss_midplane, s = 1, cmap = 'jet', norm = colors.LogNorm(vmin = 1e-15, vmax = 1e-6))
# cbar = plt.colorbar(img)
# cbar.set_label(r'$\dot{E}_{diss}$ ', fontsize = 16)
# ax.scatter(0,0,c= 'k', marker = 'x', s=80)
# ax.text(-3.5,1.5, r'E diss tot =' + str(np.round(np.sum(Edot_diss)*1e5,2)) + r'$x$1e-5', fontsize = 16)
# ax.text(-3.5,1, r'E diss mid =' + str(np.round(np.sum(EdotDiss_midplane)*1e5,2)) + r'$x$1e-5', fontsize = 16)
# ax.set_xlim(-4,0.5)
# ax.set_ylim(-2,2)
# ax.set_xlabel(r'X [$R_\odot$]', fontsize = 18)
# ax.set_ylabel(r'Y [$R_\odot$]', fontsize = 18)
# plt.title(r'Cross section, $|z|<V^{1/3}$, t/t$_{fb}$ = ' + str(np.round(tfb,3)) + f', check = {check}', fontsize = 18)

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12,4))
img = ax.scatter(X_midplane/apo, Y_midplane/apo, c = Rad_midplane, s = 1, cmap = 'jet', norm = colors.LogNorm(vmin= 1e-8, vmax = 5e-5))
cbar = plt.colorbar(img)
cbar.set_label(r'Energy density [erg/cm$^3$]', fontsize = 16)
ax.scatter(0,0,c= 'k', marker = 'x', s=80)
ax.set_xlim(-1,0.1)
ax.set_ylim(-0.5,0.5)
ax.set_xlabel(r'X [$R_\odot$]', fontsize = 18)
ax.set_ylabel(r'Y [$R_\odot$]', fontsize = 18)
plt.title(r'Cross section, $|z|<V^{1/3}$, t/t$_{fb}$ = ' + str(np.round(tfb,3)))

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (12,4))
img = ax.scatter(X_midplane/apo, Y_midplane/apo, c = L_midplane, s = 1, cmap = 'jet', norm = colors.LogNorm(vmin= 1e40, vmax = 1e46))
cbar = plt.colorbar(img)
cbar.set_label(r'Luminosity [erg/s]', fontsize = 16)
ax.scatter(0,0,c= 'k', marker = 'x', s=80)
ax.set_xlim(-8,0.1)
ax.set_ylim(-2,2)
ax.set_xlabel(r'X [$R_\odot$]', fontsize = 18)
ax.set_ylabel(r'Y [$R_\odot$]', fontsize = 18)
plt.title(r'Cross section, $|z|<V^{1/3}$, t/t$_{fb}$ = ' + str(np.round(tfb,3)))

In [ ]:
# search for indeces to explore
theta_arr, x_stream, y_stream, z_stream, thresh_cm = np.load(f'/Users/paolamartire/shocks/data/{folder}/stream_{check}{snap}.npy')
selected_indeces = np.arange(30,230,20)
plt.plot(x_stream, y_stream, c = 'k')
plt.scatter(x_stream[80], y_stream[80], c = 'k')
plt.xlim(-apo,100)
plt.ylim(-100,100)

Tangent and transverse plane

In [ ]:
idx = 80
condition_tra, x_onplane, x0 = sec.transverse_plane(X, Y, Z, dim_cell, x_stream, y_stream, z_stream, idx, coord= True)
X_tra, Y_tra, Z_tra, Vz_tra, Den_tra, Mass_tra, dim_tra = \
    sec.make_slices([X, Y, Z, vz, den, mass, dim_cell], condition_tra)
X_tra_midplane = X_tra[np.abs(Z_tra) < dim_tra]
Y_tra_midplane = Y_tra[np.abs(Z_tra) < dim_tra]

Plot as BonnerotLu22

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, figsize = (12,4))
ax1.scatter(x_onplane/Rstar, Den_tra, s=.1, c='k')
ax1.set_xlabel(r'T/R$_\star$')
ax1.set_ylabel(r'Density [M$_\odot$/R$_\odot^3$]')
ax1.set_xlim(-15,15)
ax1.grid()

ax2.scatter(Z_tra/Rstar, Vz_tra, s=.1, c='k')
ax2.set_xlabel(r'Z/R$_\star$')
ax2.set_ylabel(r'V$_z [R_\odot/\tilde{t}$]')
ax2.set_xlim(-6.5,6.5)
ax2.set_ylim(-0.5, 0.5)
ax2.grid()

plt.tight_layout()
plt.suptitle('Reproducing Fig.3 from BonnerotLu22')

In [ ]:
indeces_boundary, x_T_width, w_params, h_params, thresh = orb.find_single_boundaries(data.X, data.Y, data.Z, dim_cell, data.Mass, stream , idx, params)

In [ ]:
fig, ax = plt.subplots(1,2, figsize = (14,4), gridspec_kw = {'width_ratios': [2, 1]})
# img = ax[0].scatter(X_midplane, Y_midplane, c = Den_midplane, s = .1, cmap = 'viridis', vmin = 0.1*np.max(Den_tra), vmax = np.max(Den_tra))
# cbar = plt.colorbar(img)
# cbar.set_label(r' Density', fontsize = 16)
# ax[0].scatter(X_tra_midplane, Y_tra_midplane, s = 1, c = 'k')
# ax[0].set_xlim(-300,20)
# ax[0].set_ylim(-60,60)

img1 = ax[1].scatter(x_onplane, Z_tra, c = Den_tra,  cmap = 'viridis', s = 2, vmin = 0.1*np.max(Den_tra), vmax = np.max(Den_tra))
cbar1 = plt.colorbar(img1)
cbar1.set_label(r'Density', fontsize = 16)
ax[1].scatter(0, z_stream[idx], marker = 'x', s = 10, c = 'k', alpha = 1)
# ax[1].axvline(x_T_width[0], c = 'r')
# ax[1].axvline(x_T_width[1], c = 'r') # T coordinates for width
# ax[1].axvline(-thresh, linestyle = 'dashed', c = 'k', alpha = 0.8)
# ax[1].axvline(thresh, linestyle = 'dashed', c = 'k', alpha = 0.8)
# ax[1].text(0.9*thresh, -1.8*thresh, 'cutoff',fontsize = 11, rotation = 90,  bbox=dict(facecolor='white', alpha=0.8))
ax[1].set_xlim(-15,15)#2*thresh, 2*thresh)
ax[1].set_ylim(-15,15)#2*thresh, 2*thresh)

# ax[1].axhline(data.Z[indeces_boundary[2]], c = 'r')
# ax[1].axhline(data.Z[indeces_boundary[3]], c = 'r')
# ax[1].axhline(-thresh, linestyle = 'dashed', c = 'k', alpha = 0.8)
# ax[1].axhline(thresh, linestyle = 'dashed', c = 'k', alpha = 0.8)
# ax[1].set_ylim(-2*thresh, 2*thresh)

ax[1].set_ylabel(r'Z [$R_\odot$]', fontsize = 18) 
# plt.suptitle(f'{check}, cells W: {int(w_params[1])}, cells H: {int(h_params[1])}, theta: {np.round(theta_arr[idx], 2)}', fontsize = 16)
plt.tight_layout()


In [ ]:
len(x_onplane)